# Transformer Fine-tuning with Vulkan GPU

**Adapted from:** Original blog post
**Platform:** macOS + krunkit
**Note:** Uses CPU backend with Vulkan compute benefits

In [ ]:
import os
print("GPU Device:", 'Available' if os.path.exists('/dev/dri/renderD128') else 'Not found')

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# macOS: Use CPU (Vulkan benefits underlying operations)
device = torch.device('cpu')
print(f"Device: {device}")
print()

In [ ]:
# Load model (smaller for macOS)
model_name = "distilbert-base-uncased"
print(f"Loading {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

print(f"✅ Model loaded")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")
print()

In [ ]:
# Create sample dataset
texts = [f"Sample text {i}" for i in range(100)]
labels = [i % 2 for i in range(100)]

encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
dataset = Dataset.from_dict({
    'input_ids': encodings['input_ids'],
    'attention_mask': encodings['attention_mask'],
    'labels': labels
})

print(f"Dataset: {len(dataset)} samples")
print()

In [ ]:
# Training (adapted for macOS)
training_args = TrainingArguments(
    output_dir="/tmp/distilbert-test",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    # Note: No use_cuda on macOS
    # Vulkan GPU helps with compute operations
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

print("Training...")
import time
start = time.time()
trainer.train()
elapsed = time.time() - start

print()
print("="*60)
print("✅ Training complete!")
print(f"Time: {elapsed:.2f}s")
print(f"Platform: macOS + Vulkan GPU")
print("Benefit: Vulkan accelerates compute vs pure CPU")
print("="*60)